In [2]:
!pip install --upgrade pip
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 1.9 MB/s eta 0:00:00a 0:00:01


In [11]:
from influxdb import InfluxDBClient, DataFrameClient
from tqdm import tqdm

In [4]:
database = 'homeassistant'
username = 'homeassistant'
password = 'ko93me98'
retention_policy = 'autogen'

In [5]:
client = InfluxDBClient(host='a0d7b954-influxdb', username=username, password=password, database=database)
dfclient = DataFrameClient(host='a0d7b954-influxdb', username=username, password=password, database=database)

In [7]:
measurements = sorted([v['name'] for v in client.get_list_measurements()])
print(len(measurements))
measurements

412


['binary_sensor.abnormal_temperature',
 'binary_sensor.abnormal_voltage',
 'binary_sensor.attic_mold',
 'binary_sensor.augustinuserf_nas_drive_2_below_min_remaining_life',
 'binary_sensor.augustinuserf_nas_drive_2_exceeded_max_bad_sectors',
 'binary_sensor.augustinuserf_nas_security_status',
 'binary_sensor.augustinuserf_nas_update_available',
 'binary_sensor.backups_stale',
 'binary_sensor.bathroom_door',
 'binary_sensor.bathroom_light_overheating',
 'binary_sensor.bathroom_light_overpowering',
 'binary_sensor.bathroom_mold',
 'binary_sensor.bedroom_mold',
 'binary_sensor.bike_light_overpowering',
 'binary_sensor.bike_motion',
 'binary_sensor.bike_overheating',
 'binary_sensor.bike_socket_overpowering',
 'binary_sensor.corridor_motion',
 'binary_sensor.dark',
 'binary_sensor.desk_north_overheating',
 'binary_sensor.desk_north_overpowering',
 'binary_sensor.energy_meter_overpowering',
 'binary_sensor.experiabox_v10_wan_status',
 'binary_sensor.fridge_kitchen_overheating',
 'binary_sens

In [13]:
to_be_deleted = ['sensor.solar_temperature_3', 'sensor.solar_temperature_4']

In [30]:
to_be_changed = {measurement: measurement.replace('philips_rwl021', 'terrace') for measurement in measurements if 'philips' in measurement}
to_be_changed

{'binary_sensor.philips_rwl021_binaryinput': 'binary_sensor.terrace_binaryinput',
 'sensor.philips_rwl021_battery': 'sensor.terrace_battery'}

In [10]:
to_be_deleted = [measurement for measurement in measurements if 'flower' in measurement]
to_be_deleted

['number.flower_max_air_humidity',
 'number.flower_max_conductivity',
 'number.flower_max_dli',
 'number.flower_max_illuminance',
 'number.flower_max_soil_moisture',
 'number.flower_max_temperature',
 'number.flower_min_air_humidity',
 'number.flower_min_conductivity',
 'number.flower_min_dli',
 'number.flower_min_illuminance',
 'number.flower_min_soil_moisture',
 'number.flower_min_temperature',
 'plant.flower',
 'sensor.flower_dli',
 'sensor.flower_illuminance',
 'sensor.flower_soil_conductivity',
 'sensor.flower_soil_moisture',
 'sensor.flower_temperature',
 'sensor.flower_total_ppfd_mol_integral']

In [31]:
for old, new in tqdm(to_be_changed.items()):
    assert new != old
    try:
        dfclient.query(f'SELECT * INTO "{new}" from "{old}" GROUP BY *')
        client.drop_measurement(old)
    except:
        print(old)

  0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
for old in tqdm(to_be_deleted):
    try:
        client.drop_measurement(old)
    except:
        print(old)

100%|██████████| 2/2 [00:02<00:00,  1.07s/it]


In [8]:
measurements = [v['name'] for v in client.get_list_measurements()]
results = list()
for measurement in tqdm(measurements):
    try:
        result = list(client.query(f'SELECT * FROM "{measurement}" ORDER BY time DESC LIMIT 1').get_points())
    except:
        print(measurement)
        raise
    if len(result) == 0:
        print(measurement)
        continue
    result = result[0]
    result['entity_id'] = measurement
    results.append(result)

  0%|          | 0/412 [00:00<?, ?it/s]

In [9]:
import pandas as pd
pd.DataFrame(results).sort_values('time').head(50)

,time,device_class_str,domain,entity_id,friendly_name_str,state,value,entity_id_str,hysteresis,lower_str,...,pressure_unit_str,temperature_unit_str,visibility,visibility_unit_str,wind_bearing,wind_speed,wind_speed_unit_str,passive,persons_str,radius
197,2022-05-05T11:11:26.427775Z,NaN,sensor,sensor.flower_soil_moisture,Flower Moisture,NaN,21.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195,2022-05-05T12:07:05.498349Z,illuminance,sensor,sensor.flower_illuminance,Flower Illuminance,NaN,3291.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,2022-05-05T12:09:58.453880Z,NaN,sensor,sensor.flower_soil_conductivity,Flower Soil Conductivity,NaN,340.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
351,2022-05-05T19:16:58.015513Z,temperature,sensor,sensor.solar_temperature_3,Growatt Temperature 3,NaN,0.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352,2022-05-05T19:16:58.018731Z,temperature,sensor,sensor.solar_temperature_4,Growatt Temperature 4,NaN,0.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226,2022-07-15T19:53:01.171646Z,timestamp,sensor,sensor.last_boot,Last boot,2022-06-15T21:12:47+00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,2022-09-01T05:13:40.865044Z,temperature,sensor,sensor.flower_temperature,Flower Temperature,NaN,23.8000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
347,2022-09-09T19:46:43.693092Z,power,sensor,sensor.solar_maximum_power,Solar maximum power,NaN,3000.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,2022-09-10T18:38:29.299143Z,NaN,persistent_notification,persistent_notification.invalid_config,NaN,notifying,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,2022-09-10T21:30:14.291789Z,NaN,sensor,sensor.sqlite_web_memory_percent,None,NaN,0.3800,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
